# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm
import torch_optimizer
#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import utils
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_model import EWCLoss
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
# utils.freeze_parameters(model)

## load config and set logger

In [5]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [6]:
# Setting seed
modeler.set_seed(42)

In [7]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
PREMODEL_ROOT = config['IO_OPTION']['PREMODEL_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_FISHER_DIR = OUTPUT_ROOT + '/fisher'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_FISHER_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp2/config.yaml'

In [8]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [9]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # source mode
    dev_train_paths = dev_train_source_paths + dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(dev_train_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # source mode
    add_dev_paths = add_dev_source_paths + add_dev_target_paths
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3026, source_size : 3105, target_size : 9
Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [10]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [11]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('get fisher info mat')
    model = Model()
    premodel_path = f'{PREMODEL_ROOT}/{machine_types[0]}_model.pkl'
    pretrained_dict = torch.load(premodel_path)
    model.load_state_dict(pretrained_dict, strict=False)
    
    optimizer = torch_optimizer.RAdam(model.parameters())
    
    #ewc_loss = EWCLoss(frozen_model, fisher, lambda_ewc=1e4)
    fisher = modeler.estimate_fisher(model,
                                     dataloaders_dict['train'],
                                     sample_size=len(dev_paths[machine_type]['train']),
                                     batch_size=64)
    
    fisher_out_path = f'{OUT_FISHER_DIR}/{machine_type}_fisher.pkl'
    pd.to_pickle(fisher, fisher_out_path)
    logger.info(f'SAVE SUCCESS : {fisher_out_path}')
    
    com.toc()
    return fisher

# run

In [12]:
for machine_type in machine_types:
    fisher = run(machine_type, dev_paths)

2021-09-27 23:45:47,142 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2021-09-27 23:45:47,143 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-27 23:45:47,151 - 00_train.py - INFO - get fisher info mat
 99%|█████████▉| 94/95 [00:30<00:00,  3.08it/s]
2021-09-27 23:46:22,023 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp2/fisher/fan_fisher.pkl
2021-09-27 23:46:22,026 - 00_train.py - INFO - TARGET MACHINE_TYPE: gearbox
2021-09-27 23:46:22,026 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-27 23:46:22,029 - 00_train.py - INFO - get fisher info mat


elapsed time: 35.897284746 [sec]


 99%|█████████▉| 95/96 [00:50<00:00,  1.86it/s]
2021-09-27 23:47:13,666 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp2/fisher/gearbox_fisher.pkl
2021-09-27 23:47:13,670 - 00_train.py - INFO - TARGET MACHINE_TYPE: pump
2021-09-27 23:47:13,671 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-27 23:47:13,674 - 00_train.py - INFO - get fisher info mat


elapsed time: 51.642382145 [sec]


 99%|█████████▉| 94/95 [00:48<00:00,  1.95it/s]
2021-09-27 23:48:02,472 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp2/fisher/pump_fisher.pkl
2021-09-27 23:48:02,476 - 00_train.py - INFO - TARGET MACHINE_TYPE: slider
2021-09-27 23:48:02,477 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-27 23:48:02,479 - 00_train.py - INFO - get fisher info mat


...

elapsed time: 64.711142302 [sec]


 99%|█████████▉| 94/95 [01:24<00:00,  1.11it/s]
2021-09-27 23:51:52,442 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp2/fisher/ToyTrain_fisher.pkl
2021-09-27 23:51:52,473 - 00_train.py - INFO - TARGET MACHINE_TYPE: valve
2021-09-27 23:51:52,474 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-27 23:51:52,480 - 00_train.py - INFO - get fisher info mat


elapsed time: 85.524976969 [sec]


 99%|█████████▉| 94/95 [01:27<00:00,  1.07it/s]
2021-09-27 23:53:20,755 - 00_train.py - INFO - SAVE SUCCESS : /media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp2/fisher/valve_fisher.pkl


elapsed time: 88.293705225 [sec]


In [13]:
fisher

{'bn0.weight': tensor([6.2923e-03, 3.8733e-04, 4.9891e-04, 7.2404e-04, 9.6595e-04, 5.3799e-04,
         8.7534e-04, 3.9340e-04, 1.5914e-03, 3.1104e-04, 2.7298e-04, 2.4868e-04,
         5.5735e-04, 5.9485e-04, 4.0404e-04, 8.3576e-04, 1.2175e-03, 8.3551e-04,
         4.8641e-04, 3.2731e-04, 1.1693e-03, 5.3658e-04, 3.5196e-04, 1.5394e-03,
         1.2641e-03, 7.0331e-04, 2.1674e-03, 1.5778e-03, 3.1865e-04, 4.7773e-04,
         2.9335e-04, 2.4391e-04, 2.5141e-03, 8.6934e-04, 6.7230e-04, 5.8059e-04,
         1.1720e-03, 6.7037e-04, 4.8198e-04, 4.4670e-04, 2.6803e-04, 1.1019e-04,
         1.3489e-04, 3.8002e-04, 4.2238e-04, 4.3769e-04, 7.8198e-04, 2.2603e-04,
         1.4383e-04, 1.3320e-04, 2.5314e-04, 1.0562e-04, 8.3944e-05, 8.4606e-05,
         9.3669e-05, 6.9920e-05, 5.6296e-05, 1.1193e-04, 6.9107e-05, 1.0178e-04,
         1.6968e-04, 1.5472e-04, 1.7115e-04, 1.2041e-04, 1.1403e-04, 2.1385e-04,
         2.2563e-04, 2.4693e-04, 3.6915e-04, 1.7105e-04, 8.5331e-05, 1.2562e-04,
         6.068